# Dropzone Detection with Yolov7

**Goal of this notebook**

Train a yolov7 model on a custom dataset.  Create a program that gives instructions to align the camera with the dropzone.

**Some instructions to get this setup**

1. create conda environment with python=3.9
2. install pytorch with conda like it says to do on the official installation (use your cuda version)
* conda install pytorch torchvision torchaudio pytorch-cuda=11.7 -c pytorch -c nvidia

3. Follow this: https://www.youtube.com/watch?v=-QWxJ0j9EY8&t=378s
* skip the step where he creates the requirements_gpu step
* just run pip install -r requirements.txt instead

4. You will run into an error that is the exact same as this stack overflow post!  Follow the check marked answer and the code will work.
* https://stackoverflow.com/questions/74372636/indices-should-be-either-on-cpu-or-on-the-same-device-as-the-indexed-tensor

In [2]:
%cd yolov7

C:\Users\holla\Documents\PyTorch\UAS-club-yolov7-drone-cv-mockup-v2\yolov7


**Improvement** Instead us using --weights yolov7.pt, download the yolov7_training.pt from the transfer learning section in the repository

In [1]:
# command to train the dataset (train on cuda because of how I modified the files from the stack overflow post)
# python train.py --workers 1 --device 0 --batch-size 8 --epochs 100 --img 640 640 --data data/custom-dataset.yaml --hyp data/hyp.scratch.custom.yaml --cfg cfg/training/yolov7-custom.yaml --name yolov7-custom --weights yolov7.pt


try detect.py on the horse images given by the yolov7 repository

In [2]:
!python detect.py --weights yolov7.pt --conf 0.25 --img-size 640 --source inference/images/horses.jpg


C:\Users\holla\Documents\PyTorch\UAS-club-yolov7-drone-cv-mockup-v2\yolov7
Namespace(weights=['yolov7.pt'], source='inference/images/horses.jpg', img_size=640, conf_thres=0.25, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

5 horses, Done. (50.2ms) Inference, (6.0ms) NMS
 The image with the result is saved in: runs\detect\exp\horses.jpg
Done. (0.277s)


YOLOR  df2977d torch 1.13.0 CUDA:0 (NVIDIA GeForce GTX 1060 with Max-Q Design, 6143.8125MB)

Model Summary: 306 layers, 36905341 parameters, 6652669 gradients
C:\Users\holla\Documents\PyTorch\UAS-club-yolov7-drone-cv-mockup-v2\env\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


try detect.py on a custom dropzone image

In [4]:
!python detect.py --weights runs/train/yolov7-custom3/weights/best.pt --conf 0.4 --img-size 640 --source data/images/WIN_20221124_18_49_14_Pro.jpg --view-img --no-trace


Namespace(weights=['runs/train/yolov7-custom3/weights/best.pt'], source='data/images/WIN_20221124_18_49_14_Pro.jpg', img_size=640, conf_thres=0.4, iou_thres=0.45, device='', view_img=True, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=True)
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
1 zone 1, Done. (46.1ms) Inference, (5.5ms) NMS
 The image with the result is saved in: runs\detect\exp4\WIN_20221124_18_49_14_Pro.jpg
Done. (0.342s)


YOLOR  5a9aa0c torch 1.13.0 CUDA:0 (NVIDIA GeForce GTX 1060 with Max-Q Design, 6143.8125MB)

C:\Users\holla\Documents\PyTorch\UAS-club-yolov7-drone-cv-mockup-v2\env\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36481772 parameters, 6194944 gradients, 103.2 GFLOPS


## Get Instance of Model

article going into loading the model : https://stackabuse.com/real-time-pose-estimation-from-video-in-python-with-yolov7/

In [3]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def load_model():
    model = torch.load('runs/train/yolov7-custom3/weights/best.pt', map_location=device)['model']
    # Put in inference mode
    model.float().eval()

    if torch.cuda.is_available():
        # half() turns predictions into float16 tensors
        # which significantly lowers inference time
        model.half().to(device)
    return model

model = load_model()
model

Model(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (2): Conv(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (3): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (4): Conv(
      (conv): Conv2d(128, 64, kernel_size=(1, 1), 

## Run the code in camera-detect-drone-cv.py and detect-drone-cv.py

Youtube video explaining how to use a camera with yolov7

https://www.youtube.com/watch?v=XzUMigbYRUI

In [2]:
pwd

'C:\\Users\\holla\\Documents\\PyTorch\\UAS-club-yolov7-drone-cv-mockup-v2'

In [3]:
%cd yolov7

C:\Users\holla\Documents\PyTorch\UAS-club-yolov7-drone-cv-mockup-v2\yolov7


In [14]:
# run this in terminal: python camera-detect-done-cv.py

In [13]:

!python detect-drone-cv.py --weights runs/train/yolov7-custom3/weights/best.pt --conf 0.4 --img-size 640 --source data/images/WIN_20221124_18_49_14_Pro.jpg --view-img --no-trace


Namespace(weights=['runs/train/yolov7-custom3/weights/best.pt'], source='data/images/WIN_20221124_18_49_14_Pro.jpg', img_size=640, conf_thres=0.4, iou_thres=0.45, device='', view_img=True, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=True)
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
Shape of the image (1080, 1920, 3)
height 1080 width 1920
(1361, 209) (1520, 348)
1 zone 1, Done. (43.5ms) Inference, (13.5ms) NMS
 The image with the result is saved in: runs\detect\exp13\WIN_20221124_18_49_14_Pro.jpg
Done. (1.638s)


YOLOR  5a9aa0c torch 1.13.0 CUDA:0 (NVIDIA GeForce GTX 1060 with Max-Q Design, 6143.8125MB)

C:\Users\holla\Documents\PyTorch\UAS-club-yolov7-drone-cv-mockup-v2\env\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36481772 parameters, 6194944 gradients, 103.2 GFLOPS
